In [ ]:
# Install necessary libraries
!pip install ultralytics
!pip install plotly
!pip install opencv-python-headless

import os
import cv2
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import yaml

In [ ]:
# Paths to data directories
train_path = "/kaggle/input/brain-tumor-mri-yolov8/ready_dataset/Train"
val_path = "/kaggle/input/brain-tumor-mri-yolov8/ready_dataset/Val"

In [ ]:
# Classes
classes = ["no", "yes"]

In [ ]:
# Function to load images and labels
def load_data(data_path):
    images = []
    labels = []
    for class_label in classes:
        class_path = os.path.join(data_path, class_label, 'images')
        label_path = os.path.join(data_path, class_label, 'labels')
        for img_file in os.listdir(class_path):
            img = cv2.imread(os.path.join(class_path, img_file))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            label_file = img_file.replace('.png', '.txt')
            label_file_path = os.path.join(label_path, label_file)
            if os.path.exists(label_file_path):
                with open(label_file_path, 'r') as file:
                    label_data = file.readline().strip().split()
                    if len(label_data) > 0:
                        images.append(img)
                        labels.append(label_data)
                    else:
                        print(f"Label file {label_file_path} is empty, skipping this image.")
            else:
                print(f"Label file {label_file_path} not found, skipping this image.")
    return images, labels

# Load training and validation data
train_images, train_labels = load_data(train_path)
val_images, val_labels = load_data(val_path)

In [ ]:
# EDA - Visualize class distribution
train_counts = [len(os.listdir(os.path.join(train_path, cls, 'images'))) for cls in classes]
val_counts = [len(os.listdir(os.path.join(val_path, cls, 'images'))) for cls in classes]
eda_df = pd.DataFrame({'Class': classes, 'Train': train_counts, 'Validation': val_counts})

fig = go.Figure(data=[
    go.Bar(name='Train', x=eda_df['Class'], y=eda_df['Train']),
    go.Bar(name='Validation', x=eda_df['Class'], y=eda_df['Validation'])
])
fig.update_layout(barmode='group', title='Class Distribution in Training and Validation Sets')
fig.show()

In [ ]:
# Create YOLOv8 Dataset Configuration File
dataset_yaml = {
    'path': '/kaggle/input/brain-tumor-mri-yolov8/ready_dataset',
    'train': 'Train',
    'val': 'Val',
    'names': classes
}

with open('/kaggle/working/dataset.yaml', 'w') as file:
    yaml.dump(dataset_yaml, file)

In [ ]:
# YOLOv8 Model Training
yolo_model = YOLO("yolov8s.pt")  # Load a pre-trained YOLOv8 model
train_results = yolo_model.train(data='/kaggle/working/dataset.yaml', epochs=100, imgsz=640)

In [ ]:
# Save the YOLOv8 model
yolo_model.save('/kaggle/working/yolov8_smodel.pt')

In [ ]:
# Function to display detection results with highlighted parts
def display_samples(images, yolo_model):
    for i in range(50):
        img = images[i]
        result = yolo_model.predict(img)[0]  # Assuming batch size of 1, take the first result

        plt.figure(figsize=(4, 4))
        plt.imshow(img)
        ax = plt.gca()

        for detection in result.boxes:
            x1, y1, x2, y2 = detection.xyxy[0].cpu().numpy()
            conf = detection.conf[0].cpu().numpy()
            cls = detection.cls[0].cpu().numpy()
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            plt.text(x1, y1, f"{classes[int(cls)]} {conf:.2f}", color='white', fontsize=12, backgroundcolor='red')

        plt.title(f'YOLOv8 Detection')
        plt.show()

display_samples(val_images, yolo_model)

In [ ]:
import os

print(os.listdir('runs/detect/train3'))


In [ ]:
import pandas as pd

csv_path = 'runs/detect/train3/results.csv'  
df = pd.read_csv(csv_path)

print(df.columns)
print(df.head())
print(df.tail())

In [ ]:
import matplotlib.pyplot as plt

epochs = df.index

plt.figure(figsize=(10,5))
plt.plot(epochs, df['train/box_loss'], label='Box Loss')
plt.plot(epochs, df['train/cls_loss'], label='Class Loss')
plt.plot(epochs, df['train/dfl_loss'], label='DFL Loss')
plt.plot(epochs, df['val/box_loss'], label='Val Box Loss', linestyle='--')
plt.plot(epochs, df['val/cls_loss'], label='Val Class Loss', linestyle='--')
plt.plot(epochs, df['val/dfl_loss'], label='Val DFL Loss', linestyle='--')
plt.title("Training and Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(epochs, df['metrics/precision(B)'], label='Precision')
plt.plot(epochs, df['metrics/recall(B)'], label='Recall')
plt.plot(epochs, df['metrics/mAP50(B)'], label='mAP@0.5')
plt.plot(epochs, df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95')
plt.title("Performance Metrics")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.legend()
plt.grid()
plt.show()


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load and show confusion matrix image
img = Image.open('runs/detect/train3/confusion_matrix.png')
plt.figure(figsize=(12,8))
plt.imshow(img)
plt.axis('off')
plt.title('Confusion Matrix (Saved)')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# List of columns to plot (all metrics columns from your list)
metrics_columns = [
    "metrics/precision(B)",
    "metrics/recall(B)",
    "metrics/mAP50(B)",
    "metrics/mAP50-95(B)"
]

# Create a plot for each metric
for column in metrics_columns:
    # Plotting each metric
    plt.plot(df['epoch'], df[column], label=column)  # Use 'epoch' for x-axis

    # Adding title and axis labels for each plot
    plt.title(f"{column} over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel(f"{column} Value")
    plt.grid(True)
    plt.legend()
    
    # Show the plot
    plt.show()
